In [69]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import tree

In [52]:
#dataset 1 - 3 algorithms - 5 trials
adult_data = pd.read_csv('adult.data', header=None)
adult_data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
adult_data = adult_data.dropna()
adult_data[:10]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [3]:
def summerize_data(df):
    for column in df.columns:
        print(column)
        if df.dtypes[column] == np.object:
            print(df[column].value_counts())
        else:
            print(df[column].describe())
        print('\n')

In [53]:
summerize_data(adult_data)

age
count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64


workclass
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64


fnlwgt
count    3.256100e+04
mean     1.897784e+05
std      1.055500e+05
min      1.228500e+04
25%      1.178270e+05
50%      1.783560e+05
75%      2.370510e+05
max      1.484705e+06
Name: fnlwgt, dtype: float64


education
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctor

In [5]:
def encode(df):
    result = df.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders

In [54]:
adult_encoded, encoders = encode(adult_data)
adult_salary = adult_encoded['salary']

In [64]:
numeric_subset = adult_data.select_dtypes('number')
categorical_subset = adult_data.select_dtypes('object')
categorical_subset = pd.get_dummies(categorical_subset[categorical_subset.columns.drop('salary')])
categorical_subset[:10]

,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [60]:
adult_onehot = pd.concat([numeric_subset, categorical_subset], axis = 1)
adult_onehot[:10]

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,37,284582,14,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,49,160187,5,0,0,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,52,209642,9,0,0,45,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,31,45781,14,14084,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,42,159449,13,5178,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [56]:
adult_onehot = adult_onehot.replace({np.inf: np.nan, -np.inf: np.nan})
adult_onehot = adult_onehot.dropna()
adult_onehot[:10]

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,37,284582,14,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,49,160187,5,0,0,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,52,209642,9,0,0,45,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,31,45781,14,14084,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,42,159449,13,5178,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [101]:
#Random Forest
for i in range(5):
    print('trial', i+1)
    adult_rf_list = []
    
    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(adult_onehot, adult_salary, train_size=5000)
        
        for k in range(5):
            adult_rf = RandomForestClassifier(criterion='entropy')
            param_grid = [{'n_estimators': [1024], 'max_features': [1, 2, 4, 6, 8, 12, 16, 20]}]
            #n_list = [1.0, 2.0 , 4.0 , 6.0, 8.0, 12.0, 16.0, 20.0]
            rf_grid = GridSearchCV(adult_rf, param_grid, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            rf_grid.fit(X_train, y_train)
            print('mean test accuracy:', rf_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', rf_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', rf_grid.cv_results_['mean_test_f1_micro'])
            rf_results = pd.DataFrame(rf_grid.cv_results_['params'])
            rf_results['score_acc'] = rf_grid.cv_results_['mean_test_accuracy']
            adult_rf_list.append(rf_grid.cv_results_['mean_test_accuracy'])
            rf_cols = rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            rf_results.columns = rf_cols
            rf_results = rf_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(rf_results)

trial 1
mean test accuracy: [0.8292 0.8298 0.8356 0.8386 0.8404 0.8438 0.8444 0.8444]
mean test roc auo ovr: [0.87844948 0.88138782 0.8859192  0.8888955  0.89152628 0.89475086
 0.89713573 0.89834653]
mean test f1 micro: [0.8292 0.8298 0.8356 0.8386 0.8404 0.8438 0.8444 0.8444]
   max_features  n_estimators  score_acc
0            16          1024     0.8444
1            20          1024     0.8444
2            12          1024     0.8438
3             8          1024     0.8404
4             6          1024     0.8386
5             4          1024     0.8356
6             2          1024     0.8298
7             1          1024     0.8292
mean test accuracy: [0.8296 0.8304 0.8366 0.8384 0.842  0.845  0.8438 0.8456]
mean test roc auo ovr: [0.87783077 0.88122949 0.88555515 0.88890934 0.89154485 0.89518608
 0.89675142 0.89748326]
mean test f1 micro: [0.8296 0.8304 0.8366 0.8384 0.842  0.845  0.8438 0.8456]
   max_features  n_estimators  score_acc
0            20          1024     0.8456
1

mean test accuracy: [0.8426 0.8444 0.8496 0.8536 0.8564 0.8574 0.8586 0.8596]
mean test roc auo ovr: [0.89062263 0.89264212 0.89616435 0.89906635 0.9002688  0.90263308
 0.90381236 0.90381685]
mean test f1 micro: [0.8426 0.8444 0.8496 0.8536 0.8564 0.8574 0.8586 0.8596]
   max_features  n_estimators  score_acc
0            20          1024     0.8596
1            16          1024     0.8586
2            12          1024     0.8574
3             8          1024     0.8564
4             6          1024     0.8536
5             4          1024     0.8496
6             2          1024     0.8444
7             1          1024     0.8426
mean test accuracy: [0.8408 0.8432 0.8504 0.8544 0.8562 0.8584 0.8588 0.8594]
mean test roc auo ovr: [0.89070743 0.8929298  0.89650928 0.8990533  0.90070837 0.90249322
 0.90307294 0.90390584]
mean test f1 micro: [0.8408 0.8432 0.8504 0.8544 0.8562 0.8584 0.8588 0.8594]
   max_features  n_estimators  score_acc
0            20          1024     0.8594
1        

mean test accuracy: [0.8378 0.8426 0.8472 0.8498 0.8518 0.8536 0.8544 0.8564]
mean test roc auo ovr: [0.89055948 0.89306252 0.89744666 0.90017091 0.90240933 0.90554396
 0.9064235  0.90774043]
mean test f1 micro: [0.8378 0.8426 0.8472 0.8498 0.8518 0.8536 0.8544 0.8564]
   max_features  n_estimators  score_acc
0            20          1024     0.8564
1            16          1024     0.8544
2            12          1024     0.8536
3             8          1024     0.8518
4             6          1024     0.8498
5             4          1024     0.8472
6             2          1024     0.8426
7             1          1024     0.8378
mean test accuracy: [0.838  0.8402 0.8468 0.8478 0.8522 0.854  0.855  0.854 ]
mean test roc auo ovr: [0.8904484  0.89326567 0.8972167  0.90025113 0.90237322 0.9048899
 0.90638352 0.90763355]
mean test f1 micro: [0.838  0.8402 0.8468 0.8478 0.8522 0.854  0.855  0.854 ]
   max_features  n_estimators  score_acc
0            16          1024     0.8550
1         

mean test accuracy: [0.8344 0.8354 0.8388 0.8412 0.846  0.848  0.849  0.8494]
mean test roc auo ovr: [0.88378265 0.88653992 0.89048836 0.89330093 0.89526796 0.89858044
 0.89992756 0.900746  ]
mean test f1 micro: [0.8344 0.8354 0.8388 0.8412 0.846  0.848  0.849  0.8494]
   max_features  n_estimators  score_acc
0            20          1024     0.8494
1            16          1024     0.8490
2            12          1024     0.8480
3             8          1024     0.8460
4             6          1024     0.8412
5             4          1024     0.8388
6             2          1024     0.8354
7             1          1024     0.8344
mean test accuracy: [0.8388 0.8398 0.8426 0.8464 0.8504 0.8506 0.8548 0.8564]
mean test roc auo ovr: [0.89002357 0.89226206 0.89691667 0.90013158 0.90239638 0.90477577
 0.90658717 0.90760088]
mean test f1 micro: [0.8388 0.8398 0.8426 0.8464 0.8504 0.8506 0.8548 0.8564]
   max_features  n_estimators  score_acc
0            20          1024     0.8564
1        

mean test accuracy: [0.8398 0.84   0.8426 0.846  0.8464 0.8498 0.8516 0.8518]
mean test roc auo ovr: [0.88307883 0.88554506 0.89035937 0.89318122 0.8959085  0.89825555
 0.89984021 0.90020033]
mean test f1 micro: [0.8398 0.84   0.8426 0.846  0.8464 0.8498 0.8516 0.8518]
   max_features  n_estimators  score_acc
0            20          1024     0.8518
1            16          1024     0.8516
2            12          1024     0.8498
3             8          1024     0.8464
4             6          1024     0.8460
5             4          1024     0.8426
6             2          1024     0.8400
7             1          1024     0.8398
mean test accuracy: [0.8382 0.8416 0.8452 0.845  0.8488 0.8492 0.852  0.8516]
mean test roc auo ovr: [0.88288573 0.88648155 0.88982391 0.8935032  0.89548476 0.89849843
 0.89935048 0.90064232]
mean test f1 micro: [0.8382 0.8416 0.8452 0.845  0.8488 0.8492 0.852  0.8516]
   max_features  n_estimators  score_acc
0            16          1024     0.8520
1        

mean test accuracy: [0.8396 0.8426 0.845  0.8466 0.8482 0.8478 0.848  0.847 ]
mean test roc auo ovr: [0.89460408 0.89688793 0.90008302 0.90237152 0.90396199 0.90601191
 0.90661422 0.90760939]
mean test f1 micro: [0.8396 0.8426 0.845  0.8466 0.8482 0.8478 0.848  0.847 ]
   max_features  n_estimators  score_acc
0             8          1024     0.8482
1            16          1024     0.8480
2            12          1024     0.8478
3            20          1024     0.8470
4             6          1024     0.8466
5             4          1024     0.8450
6             2          1024     0.8426
7             1          1024     0.8396
mean test accuracy: [0.8418 0.8414 0.8402 0.8474 0.8468 0.8472 0.8464 0.846 ]
mean test roc auo ovr: [0.89430313 0.89638702 0.89974399 0.9020487  0.90369763 0.90553374
 0.9068133  0.90705607]
mean test f1 micro: [0.8418 0.8414 0.8402 0.8474 0.8468 0.8472 0.8464 0.846 ]
   max_features  n_estimators  score_acc
0             6          1024     0.8474
1        

mean test accuracy: [0.845  0.8456 0.8496 0.851  0.8524 0.8544 0.8538 0.8546]
mean test roc auo ovr: [0.89127829 0.89240863 0.89573512 0.89754232 0.89871588 0.90006454
 0.9002969  0.90100238]
mean test f1 micro: [0.845  0.8456 0.8496 0.851  0.8524 0.8544 0.8538 0.8546]
   max_features  n_estimators  score_acc
0            20          1024     0.8546
1            12          1024     0.8544
2            16          1024     0.8538
3             8          1024     0.8524
4             6          1024     0.8510
5             4          1024     0.8496
6             2          1024     0.8456
7             1          1024     0.8450
mean test accuracy: [0.844  0.847  0.8506 0.8516 0.8522 0.8538 0.8546 0.8542]
mean test roc auo ovr: [0.89099486 0.89270777 0.89543878 0.89802274 0.89892185 0.90052645
 0.90108208 0.90080033]
mean test f1 micro: [0.844  0.847  0.8506 0.8516 0.8522 0.8538 0.8546 0.8542]
   max_features  n_estimators  score_acc
0            16          1024     0.8546
1        

mean test accuracy: [0.8344 0.8376 0.842  0.8438 0.8474 0.8498 0.8512 0.8538]
mean test roc auo ovr: [0.89105298 0.89322992 0.89809679 0.90084286 0.90194709 0.90423334
 0.90528675 0.90583803]
mean test f1 micro: [0.8344 0.8376 0.842  0.8438 0.8474 0.8498 0.8512 0.8538]
   max_features  n_estimators  score_acc
0            20          1024     0.8538
1            16          1024     0.8512
2            12          1024     0.8498
3             8          1024     0.8474
4             6          1024     0.8438
5             4          1024     0.8420
6             2          1024     0.8376
7             1          1024     0.8344
mean test accuracy: [0.8336 0.8374 0.8408 0.8436 0.846  0.851  0.8518 0.8536]
mean test roc auo ovr: [0.89089717 0.89414603 0.89726103 0.90010545 0.90220312 0.90421468
 0.90502475 0.90598688]
mean test f1 micro: [0.8336 0.8374 0.8408 0.8436 0.846  0.851  0.8518 0.8536]
   max_features  n_estimators  score_acc
0            20          1024     0.8536
1        

mean test accuracy: [0.8332 0.8362 0.8402 0.8448 0.846  0.8498 0.8522 0.8524]
mean test roc auo ovr: [0.88718217 0.88991652 0.89454959 0.89800833 0.8998545  0.90319898
 0.90465694 0.90570763]
mean test f1 micro: [0.8332 0.8362 0.8402 0.8448 0.846  0.8498 0.8522 0.8524]
   max_features  n_estimators  score_acc
0            20          1024     0.8524
1            16          1024     0.8522
2            12          1024     0.8498
3             8          1024     0.8460
4             6          1024     0.8448
5             4          1024     0.8402
6             2          1024     0.8362
7             1          1024     0.8332
mean test accuracy: [0.844  0.846  0.8492 0.8512 0.8528 0.8564 0.8578 0.8584]
mean test roc auo ovr: [0.90011726 0.90211317 0.90531347 0.90737955 0.90936276 0.91140566
 0.91276741 0.91274795]
mean test f1 micro: [0.844  0.846  0.8492 0.8512 0.8528 0.8564 0.8578 0.8584]
   max_features  n_estimators  score_acc
0            20          1024     0.8584
1        

mean test accuracy: [0.8456 0.848  0.8534 0.8574 0.8588 0.8614 0.8596 0.8612]
mean test roc auo ovr: [0.89967529 0.90238865 0.90576962 0.90787937 0.91048905 0.91261544
 0.91335615 0.91377156]
mean test f1 micro: [0.8456 0.848  0.8534 0.8574 0.8588 0.8614 0.8596 0.8612]
   max_features  n_estimators  score_acc
0            12          1024     0.8614
1            20          1024     0.8612
2            16          1024     0.8596
3             8          1024     0.8588
4             6          1024     0.8574
5             4          1024     0.8534
6             2          1024     0.8480
7             1          1024     0.8456
mean test accuracy: [0.8468 0.8496 0.8546 0.8564 0.86   0.8618 0.86   0.8586]
mean test roc auo ovr: [0.90034383 0.90257566 0.90575108 0.90871929 0.91024809 0.91258058
 0.9132379  0.9139604 ]
mean test f1 micro: [0.8468 0.8496 0.8546 0.8564 0.86   0.8618 0.86   0.8586]
   max_features  n_estimators  score_acc
0            12          1024     0.8618
1        

In [102]:
adult_rf_list

[array([0.8362, 0.8364, 0.8412, 0.8434, 0.8474, 0.8492, 0.851 , 0.8508]),
 array([0.8348, 0.8358, 0.8406, 0.8442, 0.846 , 0.8484, 0.8512, 0.8532]),
 array([0.8334, 0.8364, 0.8412, 0.8434, 0.846 , 0.8494, 0.8518, 0.8516]),
 array([0.8328, 0.8362, 0.839 , 0.843 , 0.845 , 0.849 , 0.851 , 0.852 ]),
 array([0.8332, 0.8362, 0.8402, 0.8448, 0.846 , 0.8498, 0.8522, 0.8524]),
 array([0.844 , 0.846 , 0.8492, 0.8512, 0.8528, 0.8564, 0.8578, 0.8584]),
 array([0.8428, 0.8462, 0.848 , 0.8526, 0.854 , 0.8548, 0.8586, 0.857 ]),
 array([0.8444, 0.8452, 0.8476, 0.8514, 0.8524, 0.8566, 0.8584, 0.8574]),
 array([0.8426, 0.847 , 0.8478, 0.8528, 0.854 , 0.8568, 0.8574, 0.8606]),
 array([0.8426, 0.8466, 0.8504, 0.85  , 0.8528, 0.8554, 0.8558, 0.8598]),
 array([0.8386, 0.842 , 0.8432, 0.848 , 0.8502, 0.851 , 0.8538, 0.8546]),
 array([0.8412, 0.8394, 0.8452, 0.8484, 0.8498, 0.853 , 0.8528, 0.8548]),
 array([0.8412, 0.84  , 0.8448, 0.8474, 0.8498, 0.851 , 0.8516, 0.8534]),
 array([0.8392, 0.8432, 0.8452, 0.8466

In [103]:
rf_grid.cv_results_

{'mean_fit_time': array([3.97007027, 3.99928036, 4.29126406, 4.64345846, 4.95584526,
        5.19524164, 5.75541806, 6.37459497]),
 'std_fit_time': array([0.10157881, 0.03121001, 0.01540944, 0.02960937, 0.10398074,
        0.08410731, 0.03145586, 0.04873347]),
 'mean_score_time': array([0.60018702, 0.57977624, 0.5595901 , 0.53158212, 0.5159801 ,
        0.4949842 , 0.49018989, 0.47978258]),
 'std_score_time': array([0.0022238 , 0.00248428, 0.01937412, 0.00195784, 0.0055873 ,
        0.00244898, 0.0224591 , 0.01914582]),
 'param_max_features': masked_array(data=[1, 2, 4, 6, 8, 12, 16, 20],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_features': 1, 'n_estimators': 1024},


In [104]:
rf_grid.cv_results_['rank_test_accuracy']

array([8, 7, 6, 5, 4, 3, 2, 1])

In [105]:
rf_grid.cv_results_['params'][ np.argmin(rf_grid.cv_results_['rank_test_accuracy'])]

{'max_features': 20, 'n_estimators': 1024}

In [106]:
rf_grid.cv_results_['params'][ np.argmin(rf_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'max_features': 20, 'n_estimators': 1024}

In [107]:
rf_results = pd.DataFrame(rf_grid.cv_results_['params'])
rf_results['score_acc'] = rf_grid.cv_results_['mean_test_accuracy']
rf_cols = rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
rf_results.columns = rf_cols
rf_results = rf_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
rf_results

,max_features,n_estimators,score_acc
0,20,1024,0.8488
1,16,1024,0.8466
2,12,1024,0.8446
3,8,1024,0.8438
4,6,1024,0.8396
5,4,1024,0.8360
6,2,1024,0.8318
7,1,1024,0.8294


In [109]:
X_train, X_test, y_train, y_test = train_test_split(adult_onehot, adult_salary, train_size=5000)
X_train = pd.DataFrame(X_train, columns=X_train.columns)
rf_best = RandomForestClassifier(n_estimators=1024, criterion='entropy', max_features=20)
rf_best.fit(X_train, y_train)
pred_rf = rf_best.predict(X_test)
print('Random Forest:', classification_report(y_test, pred_rf))

Random Forest:               precision    recall  f1-score   support

           0       0.88      0.93      0.91     20918
           1       0.74      0.61      0.67      6643

    accuracy                           0.85     27561
   macro avg       0.81      0.77      0.79     27561
weighted avg       0.85      0.85      0.85     27561



In [111]:
#Logistic Regression
for i in range(5):
    print('trial', i+1)
    adult_logreg_list = []
    
    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(adult_onehot, adult_salary, train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)
        
        for k in range(5):
            pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', linear_model.LogisticRegression())])
            search_space = [{'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']}]
            logreg_grid = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            logreg_grid.fit(X_train, y_train)
            print('mean test accuracy:', logreg_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', logreg_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', logreg_grid.cv_results_['mean_test_f1_micro'])
            logreg_results = pd.DataFrame(logreg_grid.cv_results_['params'])
            logreg_results['score_acc'] = logreg_grid.cv_results_['mean_test_accuracy']
            adult_logreg_list.append(logreg_grid.cv_results_['mean_test_accuracy'])
            logreg_cols = logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            logreg_results.columns = logreg_cols
            logreg_results = logreg_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(logreg_results)

trial 1
mean test accuracy: [0.7682 0.7682 0.7682 0.824  0.831  0.8402 0.8482 0.8448 0.8444 0.8448
 0.844  0.844  0.844  0.844  0.844  0.844  0.844  0.844  0.7682 0.8238
 0.8398 0.8448 0.8446 0.8438 0.844  0.844  0.844  0.844  0.844 ]
mean test roc auo ovr: [0.5        0.8792428  0.5        0.88566947 0.88138146 0.89022848
 0.89792798 0.89102023 0.89236737 0.88931565 0.88921864 0.88852004
 0.88851887 0.88842453 0.88844028 0.8884122  0.88834907 0.88841669
 0.87924055 0.88514482 0.8897173  0.8910651  0.88871878 0.88612102
 0.88561888 0.88485572 0.88445473 0.88446034 0.88842454]
mean test f1 micro: [0.7682 0.7682 0.7682 0.824  0.831  0.8402 0.8482 0.8448 0.8444 0.8448
 0.844  0.844  0.844  0.844  0.844  0.844  0.844  0.844  0.7682 0.8238
 0.8398 0.8448 0.8446 0.8438 0.844  0.844  0.844  0.844  0.844 ]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8482
1   LogisticRegression(max_iter=5000) 

mean test accuracy: [0.7682 0.7682 0.7682 0.824  0.831  0.8402 0.8482 0.8448 0.8444 0.8448
 0.844  0.844  0.844  0.844  0.844  0.844  0.8442 0.844  0.7682 0.8238
 0.8398 0.8448 0.8446 0.8438 0.844  0.844  0.844  0.844  0.844 ]
mean test roc auo ovr: [0.5        0.87924168 0.5        0.88566723 0.88138033 0.89022848
 0.89792574 0.89101798 0.89236737 0.88928984 0.88920403 0.88851894
 0.88851779 0.88844701 0.88843577 0.88840205 0.88840883 0.88842455
 0.87924055 0.88514482 0.8897173  0.8910651  0.88871878 0.88612102
 0.88561888 0.88485572 0.88445473 0.88446034 0.88841558]
mean test f1 micro: [0.7682 0.7682 0.7682 0.824  0.831  0.8402 0.8482 0.8448 0.8444 0.8448
 0.844  0.844  0.844  0.844  0.844  0.844  0.8442 0.844  0.7682 0.8238
 0.8398 0.8448 0.8446 0.8438 0.844  0.844  0.844  0.844  0.844 ]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8482
1   LogisticRegression(max_iter=5000)      0.1

mean test accuracy: [0.7622 0.762  0.7622 0.8298 0.8372 0.8436 0.8488 0.8434 0.846  0.8452
 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.762  0.8302
 0.8436 0.8438 0.8444 0.8458 0.8448 0.845  0.845  0.845  0.8452]
mean test roc auo ovr: [0.5        0.88047929 0.5        0.88893764 0.8873278  0.89643952
 0.90297331 0.89872278 0.89895715 0.89746365 0.89728714 0.8970213
 0.89699815 0.89695404 0.89695182 0.89695955 0.89697058 0.89695624
 0.88047708 0.88946147 0.89643622 0.89918594 0.89605855 0.8949224
 0.89307734 0.89240791 0.89202191 0.89195684 0.89685036]
mean test f1 micro: [0.7622 0.762  0.7622 0.8298 0.8372 0.8436 0.8488 0.8434 0.846  0.8452
 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.762  0.8302
 0.8436 0.8438 0.8444 0.8458 0.8448 0.845  0.845  0.845  0.8452]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8488
1   LogisticRegression(max_iter=5000)      1.000

mean test accuracy: [0.7622 0.762  0.7622 0.8298 0.8372 0.8436 0.8488 0.8434 0.846  0.8452
 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.762  0.8302
 0.8436 0.8438 0.8444 0.8458 0.8448 0.845  0.845  0.845  0.8452]
mean test roc auo ovr: [0.5        0.8804793  0.5        0.88893763 0.8873278  0.89643842
 0.90297331 0.89874595 0.89896046 0.89746585 0.89731471 0.89702133
 0.89699595 0.89696617 0.89695183 0.89694852 0.89694742 0.8969397
 0.88047708 0.88946147 0.89643622 0.89918594 0.89605855 0.8949224
 0.89307734 0.89240791 0.89202191 0.89195684 0.89694962]
mean test f1 micro: [0.7622 0.762  0.7622 0.8298 0.8372 0.8436 0.8488 0.8434 0.846  0.8452
 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.8452 0.762  0.8302
 0.8436 0.8438 0.8444 0.8458 0.8448 0.845  0.845  0.845  0.8452]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8488
1   LogisticRegression(max_iter=5000)      1.000

mean test accuracy: [0.7548 0.7546 0.7548 0.828  0.83   0.84   0.8494 0.844  0.8452 0.845
 0.8454 0.846  0.8458 0.8458 0.8458 0.8458 0.8458 0.8458 0.7542 0.8276
 0.84   0.844  0.845  0.8454 0.8454 0.8454 0.8454 0.8454 0.8458]
mean test roc auo ovr: [0.5        0.88315689 0.5        0.89019808 0.8875869  0.89663114
 0.90483333 0.89964496 0.90049483 0.89903121 0.89904401 0.8987631
 0.89875766 0.89873821 0.89873929 0.89873929 0.89873929 0.89873281
 0.88262638 0.88966433 0.89662682 0.89963776 0.89824619 0.89797743
 0.8976521  0.89759593 0.8975711  0.89756356 0.89872205]
mean test f1 micro: [0.7548 0.7546 0.7548 0.828  0.83   0.84   0.8494 0.844  0.8452 0.845
 0.8454 0.846  0.8458 0.8458 0.8458 0.8458 0.8458 0.8458 0.7542 0.8276
 0.84   0.844  0.845  0.8454 0.8454 0.8454 0.8454 0.8454 0.8458]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8494
1   LogisticRegression(max_iter=5000)     10.0000

...

mean test accuracy: [0.7508 0.7514 0.7508 0.8208 0.8246 0.838  0.8436 0.8404 0.8416 0.841
 0.8414 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.7512 0.8206
 0.838  0.84   0.8408 0.841  0.841  0.841  0.841  0.841  0.8416]
mean test roc auo ovr: [0.5        0.87580521 0.5        0.88371752 0.87961451 0.88965493
 0.8975849  0.89276942 0.89403053 0.89259468 0.89265876 0.8924718
 0.89248143 0.89239817 0.89245471 0.89245791 0.89246434 0.89244511
 0.875002   0.88291859 0.889656   0.89197158 0.89146983 0.89049536
 0.89082399 0.8906636  0.89063258 0.89063258 0.89245365]
mean test f1 micro: [0.7508 0.7514 0.7508 0.8208 0.8246 0.838  0.8436 0.8404 0.8416 0.841
 0.8414 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.7512 0.8206
 0.838  0.84   0.8408 0.841  0.841  0.841  0.841  0.841  0.8416]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8436
1   LogisticRegression(max_iter=5000)   1000.0000

mean test accuracy: [0.7508 0.7514 0.7508 0.8208 0.8246 0.838  0.8436 0.8404 0.8416 0.841
 0.8414 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.7512 0.8206
 0.838  0.84   0.8408 0.841  0.841  0.841  0.841  0.841  0.8416]
mean test roc auo ovr: [0.5        0.87580414 0.5        0.88371539 0.87961451 0.88965599
 0.8975849  0.89277049 0.8940316  0.89260857 0.89265662 0.89247928
 0.8924825  0.89245792 0.89245793 0.89244938 0.89245579 0.89246003
 0.875002   0.88291859 0.889656   0.89197158 0.89146983 0.89049536
 0.89082399 0.8906636  0.89063258 0.89063258 0.89245578]
mean test f1 micro: [0.7508 0.7514 0.7508 0.8208 0.8246 0.838  0.8436 0.8404 0.8416 0.841
 0.8414 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.8416 0.7512 0.8206
 0.838  0.84   0.8408 0.841  0.841  0.841  0.841  0.841  0.8416]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8436
1   LogisticRegression(max_iter=5000)   1000.000

mean test accuracy: [0.7654 0.7656 0.7654 0.8302 0.8342 0.8478 0.8556 0.8556 0.853  0.8534
 0.8536 0.8534 0.8536 0.8536 0.8536 0.8536 0.8536 0.8536 0.7656 0.8298
 0.8474 0.8556 0.8532 0.853  0.8536 0.8536 0.8536 0.8536 0.8536]
mean test roc auo ovr: [0.5        0.88586367 0.5        0.89516487 0.88878604 0.9010906
 0.90966069 0.90588708 0.90771489 0.90587797 0.90598706 0.90547221
 0.90545549 0.90538084 0.90537639 0.90536192 0.90536971 0.90537192
 0.88586367 0.89516376 0.90056883 0.90530237 0.90580989 0.90267552
 0.90412044 0.90395097 0.90383938 0.90378023 0.9053697 ]
mean test f1 micro: [0.7654 0.7656 0.7654 0.8302 0.8342 0.8478 0.8556 0.8556 0.853  0.8534
 0.8536 0.8534 0.8536 0.8536 0.8536 0.8536 0.8536 0.8536 0.7656 0.8298
 0.8474 0.8556 0.8532 0.853  0.8536 0.8536 0.8536 0.8536 0.8536]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8556
1   LogisticRegression(max_iter=5000)      0.10

mean test accuracy: [0.7602 0.7602 0.7602 0.825  0.832  0.835  0.8444 0.8404 0.8408 0.8406
 0.8406 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.76   0.825
 0.8346 0.8402 0.8406 0.8406 0.8404 0.8406 0.8406 0.8406 0.8404]
mean test roc auo ovr: [0.5        0.87551548 0.5        0.88254979 0.88182367 0.88874054
 0.89583849 0.89088088 0.89207231 0.88967627 0.88967742 0.88922851
 0.88922631 0.88917911 0.88917691 0.88916813 0.88917803 0.88916703
 0.87525197 0.88254212 0.88821752 0.89062504 0.88914426 0.88727971
 0.88628653 0.88589696 0.8857237  0.88571162 0.88916814]
mean test f1 micro: [0.7602 0.7602 0.7602 0.825  0.832  0.835  0.8444 0.8404 0.8408 0.8406
 0.8406 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.76   0.825
 0.8346 0.8402 0.8406 0.8406 0.8404 0.8406 0.8406 0.8406 0.8404]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8444
1   LogisticRegression(max_iter=5000)      1.000

mean test accuracy: [0.7602 0.7602 0.7602 0.825  0.832  0.835  0.8444 0.8404 0.8408 0.8406
 0.8406 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.76   0.825
 0.8346 0.8402 0.8406 0.8406 0.8404 0.8406 0.8406 0.8406 0.8404]
mean test roc auo ovr: [0.5        0.87551329 0.5        0.88255089 0.88182367 0.88874164
 0.8958374  0.89087978 0.8920734  0.88967407 0.88966973 0.8892428
 0.88922084 0.88917801 0.88917144 0.88916594 0.88917363 0.88916704
 0.87525197 0.88254212 0.88821752 0.89062504 0.88914426 0.88727971
 0.88628653 0.88589696 0.8857237  0.88571162 0.88917692]
mean test f1 micro: [0.7602 0.7602 0.7602 0.825  0.832  0.835  0.8444 0.8404 0.8408 0.8406
 0.8406 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.8404 0.76   0.825
 0.8346 0.8402 0.8406 0.8406 0.8404 0.8406 0.8406 0.8406 0.8404]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.8444
1   LogisticRegression(max_iter=5000)      1.0000

In [119]:
adult_logreg_list

[array([0.7502, 0.7506, 0.7502, 0.8208, 0.8254, 0.8372, 0.8426, 0.8432,
        0.844 , 0.845 , 0.8446, 0.8444, 0.8444, 0.8444, 0.8444, 0.8444,
        0.8444, 0.8444, 0.7506, 0.821 , 0.8372, 0.8432, 0.8448, 0.8444,
        0.8446, 0.8446, 0.8446, 0.8446, 0.8444]),
 array([0.7502, 0.7506, 0.7502, 0.8208, 0.8254, 0.8372, 0.8426, 0.8432,
        0.844 , 0.845 , 0.8446, 0.8444, 0.8444, 0.8444, 0.8444, 0.8444,
        0.8444, 0.8444, 0.7506, 0.821 , 0.8372, 0.8432, 0.8448, 0.8444,
        0.8446, 0.8446, 0.8446, 0.8446, 0.8444]),
 array([0.7502, 0.7506, 0.7502, 0.8208, 0.8254, 0.8372, 0.8426, 0.8432,
        0.844 , 0.845 , 0.8446, 0.8444, 0.8444, 0.8444, 0.8444, 0.8444,
        0.8444, 0.8444, 0.7506, 0.821 , 0.8372, 0.8432, 0.8448, 0.8444,
        0.8446, 0.8446, 0.8446, 0.8446, 0.8444]),
 array([0.7502, 0.7506, 0.7502, 0.8208, 0.8254, 0.8372, 0.8426, 0.8432,
        0.844 , 0.8448, 0.8446, 0.8444, 0.8444, 0.8444, 0.8444, 0.8444,
        0.8444, 0.8444, 0.7506, 0.821 , 0.8372, 0.8432, 0.

In [112]:
logreg_grid.cv_results_

{'mean_fit_time': array([ 0.0272016 ,  0.19839382,  0.05079908,  0.34379282,  0.87417903,
         0.94077625,  2.93411336,  3.49889336, 14.82353082,  2.88011231,
         3.29389906,  2.89690971,  3.36209679,  2.94850655,  3.43409433,
         2.9655036 ,  3.43489194,  2.94571066,  0.02860184,  0.02907195,
         0.03499618,  0.05094495,  0.07038355,  0.10626822,  0.08104458,
         0.08522997,  0.08938751,  0.09069343,  2.60451736]),
 'std_fit_time': array([2.39840167e-03, 2.86239196e-02, 2.40563392e-03, 1.16208133e-01,
        1.10575993e-01, 5.86170024e-01, 8.97229732e-01, 2.64280574e+00,
        1.13411823e+01, 1.17979523e+00, 7.20912251e-01, 6.27423425e-01,
        6.87207564e-01, 6.09401858e-01, 6.97364705e-01, 5.92709278e-01,
        7.05964889e-01, 6.13816826e-01, 4.86643319e-04, 9.82944323e-04,
        2.00602119e-03, 3.58633356e-03, 1.27282459e-02, 3.79086079e-02,
        1.09741849e-02, 1.51864563e-02, 1.24687836e-02, 1.91512260e-02,
        5.38126118e-01]),
 'mean_sco

In [113]:
logreg_grid.cv_results_['rank_test_accuracy']

array([26, 26, 26, 24, 23, 21,  1, 10,  2,  6,  6, 10, 10, 10, 10, 10, 10,
       10, 29, 24, 22, 20,  6,  6, 10,  3,  3,  3, 10])

In [114]:
logreg_grid.cv_results_['params'][ np.argmin(logreg_grid.cv_results_['rank_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [115]:
logreg_grid.cv_results_['params'][ np.argmin(logreg_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [116]:
logreg_results = pd.DataFrame(logreg_grid.cv_results_['params'])
logreg_results['score_acc'] = logreg_grid.cv_results_['mean_test_accuracy']
logreg_cols = logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
logreg_results.columns = logreg_cols
logreg_results = logreg_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
logreg_results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=5000),0.1000,l1,saga,0.8444
1,LogisticRegression(max_iter=5000),1.0000,l1,saga,0.8408
2,LogisticRegression(max_iter=5000),NaN,none,lbfgs,0.8406
3,LogisticRegression(max_iter=5000),10000.0000,l2,lbfgs,0.8406
4,LogisticRegression(max_iter=5000),1000.0000,l2,lbfgs,0.8406
5,LogisticRegression(max_iter=5000),10.0000,l2,lbfgs,0.8406
6,LogisticRegression(max_iter=5000),1.0000,l2,saga,0.8406
7,LogisticRegression(max_iter=5000),10.0000,l1,saga,0.8406
8,LogisticRegression(max_iter=5000),1.0000,l2,lbfgs,0.8406
9,LogisticRegression(max_iter=5000),1000.0000,l1,saga,0.8404


In [117]:
X_train, X_test, y_train, y_test = train_test_split(adult_onehot, adult_salary, train_size=5000)
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = sc.transform(X_test)
logreg_best = linear_model.LogisticRegression(penalty='l1', C=0.1, solver='saga', max_iter=5000)
logreg_best.fit(X_train, y_train)
pred_logreg = logreg_best.predict(X_test)
print('Logistic Regression:', classification_report(y_test, pred_logreg))

Logistic Regression:               precision    recall  f1-score   support

           0       0.88      0.94      0.91     20941
           1       0.74      0.58      0.65      6620

    accuracy                           0.85     27561
   macro avg       0.81      0.76      0.78     27561
weighted avg       0.84      0.85      0.85     27561



In [120]:
# KNN
for i in range(5):
    print('trial', i+1)
    adult_knn_list = []
    
    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(adult_onehot, adult_salary, train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)
        
        for k in range(5):
            adult_knn = KNeighborsClassifier(weights='distance')
            k_list = [1, 5 , 9 , 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101]
            knn_grid = GridSearchCV(adult_knn, {'n_neighbors':k_list}, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            knn_grid.fit(X_train, y_train)
            print('mean test accuracy:', knn_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', knn_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', knn_grid.cv_results_['mean_test_f1_micro'])
            knn_results = pd.DataFrame(knn_grid.cv_results_['params'])
            knn_results['score_acc'] = knn_grid.cv_results_['mean_test_accuracy']
            adult_knn_list.append(knn_grid.cv_results_['mean_test_accuracy'])
            knn_cols = knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            knn_results.columns = knn_cols
            knn_results = knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(knn_results)

trial 1
mean test accuracy: [0.773  0.7988 0.8066 0.8088 0.8102 0.8124 0.8162 0.8136 0.817  0.8184
 0.8198 0.8218 0.8242 0.8238 0.8248 0.827  0.8258 0.8262 0.8272 0.8282
 0.828  0.8278 0.8284 0.8286 0.8278 0.8254]
mean test roc auo ovr: [0.7023692  0.82761777 0.84735191 0.85315679 0.85808026 0.86177443
 0.86504225 0.86783221 0.86972259 0.87166564 0.87327082 0.8747299
 0.87588092 0.87659347 0.8772851  0.87734147 0.87792907 0.87835702
 0.87911517 0.87936473 0.87996349 0.88029356 0.8810508  0.88093409
 0.88149764 0.88163411]
mean test f1 micro: [0.773  0.7988 0.8066 0.8088 0.8102 0.8124 0.8162 0.8136 0.817  0.8184
 0.8198 0.8218 0.8242 0.8238 0.8248 0.827  0.8258 0.8262 0.8272 0.8282
 0.828  0.8278 0.8284 0.8286 0.8278 0.8254]
    n_neighbors  score_acc
0            93     0.8286
1            89     0.8284
2            77     0.8282
3            81     0.8280
4            97     0.8278
5            85     0.8278
6            73     0.8272
7            61     0.8270
8            69     0.8

mean test accuracy: [0.777  0.8096 0.8198 0.8246 0.8226 0.8232 0.8262 0.83   0.8326 0.8318
 0.832  0.8302 0.8332 0.8322 0.833  0.834  0.8338 0.8338 0.8324 0.8348
 0.834  0.8338 0.8326 0.8358 0.8336 0.8336]
mean test roc auo ovr: [0.69175682 0.8184085  0.84177552 0.85243441 0.85747804 0.86044795
 0.86252118 0.8647565  0.8662624  0.86799233 0.86994807 0.8700049
 0.87067517 0.87135659 0.87146685 0.87140444 0.87151511 0.87179892
 0.87204646 0.87224868 0.87191282 0.8722515  0.87241119 0.87259781
 0.87288726 0.8732764 ]
mean test f1 micro: [0.777  0.8096 0.8198 0.8246 0.8226 0.8232 0.8262 0.83   0.8326 0.8318
 0.832  0.8302 0.8332 0.8322 0.833  0.834  0.8338 0.8338 0.8324 0.8348
 0.834  0.8338 0.8326 0.8358 0.8336 0.8336]
    n_neighbors  score_acc
0            93     0.8358
1            77     0.8348
2            81     0.8340
3            61     0.8340
4            85     0.8338
5            69     0.8338
6            65     0.8338
7           101     0.8336
8            97     0.8336
9   

mean test accuracy: [0.776  0.8072 0.814  0.8182 0.8222 0.8236 0.8196 0.8212 0.8254 0.8258
 0.8274 0.8296 0.827  0.8254 0.8262 0.8246 0.8248 0.8266 0.826  0.8244
 0.8244 0.8232 0.8236 0.8248 0.8264 0.8274]
mean test roc auo ovr: [0.69031858 0.81571523 0.83684435 0.84816376 0.85452826 0.8581178
 0.86144411 0.8635057  0.86491863 0.8665815  0.86824604 0.86913969
 0.86984754 0.8699139  0.8702186  0.87076386 0.87101437 0.87103539
 0.87110894 0.87111557 0.87159834 0.8721801  0.87263356 0.87289292
 0.87337514 0.87366878]
mean test f1 micro: [0.776  0.8072 0.814  0.8182 0.8222 0.8236 0.8196 0.8212 0.8254 0.8258
 0.8274 0.8296 0.827  0.8254 0.8262 0.8246 0.8248 0.8266 0.826  0.8244
 0.8244 0.8232 0.8236 0.8248 0.8264 0.8274]
    n_neighbors  score_acc
0            45     0.8296
1            41     0.8274
2           101     0.8274
3            49     0.8270
4            69     0.8266
5            97     0.8264
6            57     0.8262
7            73     0.8260
8            37     0.8258
9   

mean test accuracy: [0.7762 0.805  0.8088 0.8146 0.8194 0.8174 0.8204 0.825  0.8262 0.8246
 0.8238 0.8234 0.822  0.8224 0.8224 0.823  0.8228 0.8234 0.8248 0.8272
 0.8258 0.826  0.8266 0.8264 0.8262 0.8268]
mean test roc auo ovr: [0.69517656 0.81864949 0.84218819 0.85291003 0.85894255 0.86305991
 0.86661692 0.86907082 0.87059432 0.87199693 0.87332919 0.8738435
 0.87424616 0.87478203 0.8748417  0.87560368 0.875429   0.87579511
 0.87639415 0.8766337  0.87699888 0.87756152 0.87795892 0.87826565
 0.878552   0.87900847]
mean test f1 micro: [0.7762 0.805  0.8088 0.8146 0.8194 0.8174 0.8204 0.825  0.8262 0.8246
 0.8238 0.8234 0.822  0.8224 0.8224 0.823  0.8228 0.8234 0.8248 0.8272
 0.8258 0.826  0.8266 0.8264 0.8262 0.8268]
    n_neighbors  score_acc
0            77     0.8272
1           101     0.8268
2            89     0.8266
3            93     0.8264
4            97     0.8262
5            33     0.8262
6            85     0.8260
7            81     0.8258
8            29     0.8250
9   

mean test accuracy: [0.7748 0.805  0.8164 0.8158 0.8174 0.8192 0.8208 0.8216 0.8244 0.8252
 0.8254 0.8256 0.8248 0.8246 0.8234 0.823  0.8226 0.822  0.8228 0.823
 0.8232 0.8246 0.8244 0.8228 0.824  0.8242]
mean test roc auo ovr: [0.69147946 0.81747344 0.84420757 0.85420711 0.86113721 0.86447915
 0.86660428 0.86972265 0.8714252  0.87249008 0.87341013 0.87438471
 0.87491045 0.87582471 0.87613394 0.876576   0.87683641 0.87680566
 0.87691067 0.8769784  0.87651241 0.87679089 0.87722913 0.8770763
 0.87753044 0.87783031]
mean test f1 micro: [0.7748 0.805  0.8164 0.8158 0.8174 0.8192 0.8208 0.8216 0.8244 0.8252
 0.8254 0.8256 0.8248 0.8246 0.8234 0.823  0.8226 0.822  0.8228 0.823
 0.8232 0.8246 0.8244 0.8228 0.824  0.8242]
    n_neighbors  score_acc
0            45     0.8256
1            41     0.8254
2            37     0.8252
3            49     0.8248
4            53     0.8246
5            85     0.8246
6            33     0.8244
7            89     0.8244
8           101     0.8242
9     

...

mean test accuracy: [0.7874 0.815  0.822  0.8298 0.8324 0.8328 0.8314 0.8338 0.8316 0.833
 0.832  0.8328 0.8326 0.8324 0.8328 0.8328 0.8336 0.8322 0.8328 0.8332
 0.8322 0.8328 0.8326 0.8304 0.8298 0.8314]
mean test roc auo ovr: [0.71024    0.83164434 0.84958781 0.8599451  0.86472895 0.86881425
 0.87223894 0.87421981 0.87674451 0.87770689 0.87795915 0.87876348
 0.87982382 0.88043834 0.88068293 0.88051602 0.88052506 0.88041962
 0.88073799 0.88148349 0.88162042 0.88188602 0.88170223 0.88179918
 0.88172866 0.88156013]
mean test f1 micro: [0.7874 0.815  0.822  0.8298 0.8324 0.8328 0.8314 0.8338 0.8316 0.833
 0.832  0.8328 0.8326 0.8324 0.8328 0.8328 0.8336 0.8322 0.8328 0.8332
 0.8322 0.8328 0.8326 0.8304 0.8298 0.8314]
    n_neighbors  score_acc
0            29     0.8338
1            65     0.8336
2            77     0.8332
3            37     0.8330
4            85     0.8328
5            21     0.8328
6            61     0.8328
7            57     0.8328
8            73     0.8328
9    

mean test accuracy: [0.7946 0.817  0.8216 0.8252 0.8284 0.828  0.829  0.8328 0.8328 0.833
 0.8344 0.8354 0.8372 0.8374 0.8384 0.8386 0.8388 0.8388 0.8418 0.8404
 0.84   0.8374 0.8364 0.8368 0.8358 0.8348]
mean test roc auo ovr: [0.70696136 0.81594193 0.84003989 0.8489802  0.85574066 0.860041
 0.86188425 0.86352467 0.86520748 0.86703932 0.86734456 0.86818161
 0.86903975 0.86976358 0.87011901 0.87044835 0.87123331 0.87184975
 0.87157793 0.87156603 0.87165971 0.87203149 0.8721533  0.87244609
 0.87198669 0.87269629]
mean test f1 micro: [0.7946 0.817  0.8216 0.8252 0.8284 0.828  0.829  0.8328 0.8328 0.833
 0.8344 0.8354 0.8372 0.8374 0.8384 0.8386 0.8388 0.8388 0.8418 0.8404
 0.84   0.8374 0.8364 0.8368 0.8358 0.8348]
    n_neighbors  score_acc
0            73     0.8418
1            77     0.8404
2            81     0.8400
3            65     0.8388
4            69     0.8388
5            61     0.8386
6            57     0.8384
7            53     0.8374
8            85     0.8374
9      

mean test accuracy: [0.78   0.8114 0.8194 0.8226 0.8282 0.8282 0.8286 0.8314 0.8322 0.8282
 0.8284 0.8272 0.8286 0.828  0.8278 0.83   0.8284 0.828  0.8296 0.8288
 0.8288 0.8284 0.8282 0.828  0.8266 0.8282]
mean test roc auo ovr: [0.70070565 0.82532979 0.8413709  0.84975636 0.85573397 0.85831656
 0.86120827 0.86327233 0.86468029 0.86640044 0.86806542 0.86952473
 0.87033887 0.87117363 0.87211939 0.87252017 0.87278536 0.87300564
 0.87341937 0.87354769 0.87344422 0.87390611 0.87410134 0.87437551
 0.87481177 0.87516159]
mean test f1 micro: [0.78   0.8114 0.8194 0.8226 0.8282 0.8282 0.8286 0.8314 0.8322 0.8282
 0.8284 0.8272 0.8286 0.828  0.8278 0.83   0.8284 0.828  0.8296 0.8288
 0.8288 0.8284 0.8282 0.828  0.8266 0.8282]
    n_neighbors  score_acc
0            33     0.8322
1            29     0.8314
2            61     0.8300
3            73     0.8296
4            77     0.8288
5            81     0.8288
6            25     0.8286
7            49     0.8286
8            41     0.8284
9  

mean test accuracy: [0.7848 0.8104 0.8186 0.8232 0.8224 0.8274 0.825  0.8292 0.8324 0.8326
 0.833  0.8314 0.8318 0.8304 0.8306 0.8302 0.8322 0.8336 0.8316 0.8312
 0.8312 0.8298 0.8286 0.83   0.8314 0.8302]
mean test roc auo ovr: [0.69957585 0.82002975 0.84204644 0.85278852 0.85875541 0.86348027
 0.86814761 0.86919831 0.87099944 0.87223098 0.87357588 0.87543618
 0.87633868 0.87698569 0.87747234 0.87798331 0.87835493 0.87805962
 0.87793575 0.878074   0.87876304 0.8791225  0.87931383 0.87959697
 0.88028269 0.8804309 ]
mean test f1 micro: [0.7848 0.8104 0.8186 0.8232 0.8224 0.8274 0.825  0.8292 0.8324 0.8326
 0.833  0.8314 0.8318 0.8304 0.8306 0.8302 0.8322 0.8336 0.8316 0.8312
 0.8312 0.8298 0.8286 0.83   0.8314 0.8302]
    n_neighbors  score_acc
0            69     0.8336
1            41     0.8330
2            37     0.8326
3            33     0.8324
4            65     0.8322
5            49     0.8318
6            73     0.8316
7            97     0.8314
8            45     0.8314
9  

mean test accuracy: [0.7834 0.8084 0.814  0.8248 0.826  0.8282 0.8258 0.8296 0.8318 0.8302
 0.829  0.8306 0.8306 0.831  0.8318 0.8328 0.8342 0.8318 0.8328 0.8328
 0.8322 0.831  0.8308 0.83   0.829  0.8306]
mean test roc auo ovr: [0.70953947 0.82382018 0.84770724 0.85794298 0.86282237 0.86579441
 0.86805702 0.87020998 0.87218037 0.8737489  0.87531963 0.87623849
 0.87713487 0.878625   0.87969298 0.8802193  0.88095833 0.88130154
 0.88135746 0.88136952 0.88159978 0.8819364  0.88199013 0.88177412
 0.88206469 0.88229605]
mean test f1 micro: [0.7834 0.8084 0.814  0.8248 0.826  0.8282 0.8258 0.8296 0.8318 0.8302
 0.829  0.8306 0.8306 0.831  0.8318 0.8328 0.8342 0.8318 0.8328 0.8328
 0.8322 0.831  0.8308 0.83   0.829  0.8306]
    n_neighbors  score_acc
0            65     0.8342
1            77     0.8328
2            73     0.8328
3            61     0.8328
4            81     0.8322
5            57     0.8318
6            69     0.8318
7            33     0.8318
8            53     0.8310
9  

In [121]:
adult_knn_list

[array([0.7946, 0.817 , 0.8216, 0.8252, 0.8284, 0.828 , 0.829 , 0.8328,
        0.8328, 0.833 , 0.8344, 0.8354, 0.8372, 0.8374, 0.8384, 0.8386,
        0.8388, 0.8388, 0.8418, 0.8404, 0.84  , 0.8374, 0.8364, 0.8368,
        0.8358, 0.8348]),
 array([0.7946, 0.817 , 0.8216, 0.8252, 0.8284, 0.828 , 0.829 , 0.8328,
        0.8328, 0.833 , 0.8344, 0.8354, 0.8372, 0.8374, 0.8384, 0.8386,
        0.8388, 0.8388, 0.8418, 0.8404, 0.84  , 0.8374, 0.8364, 0.8368,
        0.8358, 0.8348]),
 array([0.7946, 0.817 , 0.8216, 0.8252, 0.8284, 0.828 , 0.829 , 0.8328,
        0.8328, 0.833 , 0.8344, 0.8354, 0.8372, 0.8374, 0.8384, 0.8386,
        0.8388, 0.8388, 0.8418, 0.8404, 0.84  , 0.8374, 0.8364, 0.8368,
        0.8358, 0.8348]),
 array([0.7946, 0.817 , 0.8216, 0.8252, 0.8284, 0.828 , 0.829 , 0.8328,
        0.8328, 0.833 , 0.8344, 0.8354, 0.8372, 0.8374, 0.8384, 0.8386,
        0.8388, 0.8388, 0.8418, 0.8404, 0.84  , 0.8374, 0.8364, 0.8368,
        0.8358, 0.8348]),
 array([0.7946, 0.817 , 0.8216, 

In [122]:
knn_grid.cv_results_

{'mean_fit_time': array([0.00580626, 0.00540075, 0.00539799, 0.00538783, 0.00559816,
        0.00559702, 0.00499182, 0.00540023, 0.005199  , 0.00539684,
        0.0049983 , 0.00519714, 0.00540328, 0.0053937 , 0.00560389,
        0.00539632, 0.00579691, 0.00520358, 0.00559864, 0.00519695,
        0.00540686, 0.00560017, 0.0053947 , 0.00579896, 0.00580039,
        0.00519996]),
 'std_fit_time': array([4.03121230e-04, 4.89999789e-04, 4.91683888e-04, 4.93860348e-04,
        4.88030674e-04, 4.87595259e-04, 7.38620604e-06, 4.85389461e-04,
        4.00523469e-04, 4.92442710e-04, 1.05354038e-05, 4.01844669e-04,
        4.93709512e-04, 4.88611604e-04, 4.91799628e-04, 4.87573733e-04,
        1.16570387e-03, 3.98027478e-04, 8.00540795e-04, 3.94392086e-04,
        4.90400787e-04, 4.89940803e-04, 4.95166725e-04, 3.99518109e-04,
        4.00257111e-04, 3.99685162e-04]),
 'mean_score_time': array([0.13479357, 0.19199357, 0.19859672, 0.19359665, 0.19899616,
        0.19539394, 0.19539647, 0.19578385, 

In [123]:
knn_grid.cv_results_['rank_test_accuracy']

array([26, 25, 24, 23, 21, 20, 22, 17,  6, 15, 18, 13, 13,  9,  6,  2,  1,
        6,  2,  2,  5,  9, 11, 16, 18, 12])

In [124]:
knn_grid.cv_results_['params'][ np.argmin(knn_grid.cv_results_['rank_test_accuracy'])]

{'n_neighbors': 65}

In [125]:
knn_grid.cv_results_['params'][ np.argmin(knn_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'n_neighbors': 101}

In [126]:
knn_results = pd.DataFrame(knn_grid.cv_results_['params'])
knn_results['score_acc'] = knn_grid.cv_results_['mean_test_accuracy']
knn_cols = knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
knn_results.columns = knn_cols
knn_results = knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
knn_results

,n_neighbors,score_acc
0,65,0.8342
1,77,0.8328
2,73,0.8328
3,61,0.8328
4,81,0.8322
5,57,0.8318
6,69,0.8318
7,33,0.8318
8,53,0.8310
9,85,0.8310


In [127]:
X_train, X_test, y_train, y_test = train_test_split(adult_onehot, adult_salary, train_size=5000)
X_train = pd.DataFrame(X_train, columns=X_train.columns)
knn_best = KNeighborsClassifier(weights='distance', n_neighbors=101)
knn_best.fit(X_train, y_train)
pred_knn = knn_best.predict(X_test)
print('KNN:', classification_report(y_test, pred_knn))

KNN:               precision    recall  f1-score   support

           0       0.77      0.97      0.86     20942
           1       0.54      0.10      0.16      6619

    accuracy                           0.76     27561
   macro avg       0.66      0.54      0.51     27561
weighted avg       0.72      0.76      0.69     27561

